In [ ]:
import numpy as np
import pandas as pd
from SGsmooth import SavitzkyGolaySmooth #import SG-smooth func.
import matplotlib.pyplot as plt
import os

## open files

In [ ]:
f_data = pd.read_csv('./data/20151204-variability of saliva_final55plex_Healthy_chromatogame.tsv',sep='\t')
f_summary = pd.read_csv('./data/Transition Results-variability of saliva_final55plex_Healthy_20170413-2_boundary.csv')

In [ ]:
FileName = np.unique(f_summary['File Name'])
PeptideSequence = np.unique(f_summary['Peptide Sequence'])

## check output directory

In [ ]:
if 'MRM_RT_rearrangement' not in os.listdir():
    os.mkdir('MRM_RT_rearrangement')
if 'test' not in os.listdir('MRM_RT_rearrangement'):
    os.mkdir('MRM_RT_rearrangement/test')

## rearrange the boundary for each datum

In [ ]:
for fn in FileName:
    for ps in PeptideSequence:
        try:
            f_data_sub = f_data[(f_data['FileName']==fn)&(f_data['PeptideModifiedSequence']==ps)]
            #Fragmention: 特定file name & peptide sequence的"FragmentIon"
            Fragmention = np.unique(f_data_sub['FragmentIon'])

            #先處理heavy 
            intHeavy = [] #intHeavy: intensity of heavy part signal，應放入3個不同fragment ion的Intensities陣列
            timeHeavy = [] #timeHeavy: retention time of heavy part signal，應放入3個不同fragment ion的Times陣列
            for f in Fragmention:
                intHeavy.append(list(map(float,list(f_data_sub[(f_data_sub['FragmentIon']==f) & (f_data_sub['IsotopeLabelType']=='heavy')]['Intensities'])[0].split(','))))
                timeHeavy.append(list(map(float,list(f_data_sub[(f_data_sub['FragmentIon']==f) & (f_data_sub['IsotopeLabelType']=='heavy')]['Times'])[0].split(','))))

            ########################################################################################
            #對訊號intensity做SG-Smooth
            intHeavy_smooth = [SavitzkyGolaySmooth(i) for i in intHeavy]
            #呼叫原始程式所決定的boundary，分為start及end
            start = list(f_summary[(f_summary['File Name']==fn) & (f_summary['Peptide Sequence']==ps)]['heavy Start Time'])[0]
            end = list(f_summary[(f_summary['File Name']==fn) & (f_summary['Peptide Sequence']==ps)]['heavy End Time'])[0]
            #peak_max: 三個不同fragment ion的最高intensity數值
            #peak_index: 三個不同fragment ion的最高intensity數值之index
            #peak_time: 三個不同fragment ion的最高intensity數值出現的時間
            peak_max = [max(np.array(intHeavy_smooth[i])[(start<np.array(timeHeavy[i]))&(np.array(timeHeavy[i])<end)]) for i in range(len(intHeavy_smooth))]
            peak_index = [intHeavy_smooth[i].index(peak_max[i]) for i in range(len(peak_max))]
            peak_time = [timeHeavy[i][peak_index[i]] for i in range(len(peak_index))]
            #max_peak_time_in3: 三個peak之最高點的最右側時間
            #max_peak_time_in3: 三個peak之最高點的最左側時間
            #center: 原始程式框好的boundary之中心的時間 
            max_peak_time_in3 = max(peak_time)
            min_peak_time_in3 = min(peak_time)
            center = (start+end)/2
            #timeHeavy_cutoff: 所有時間點間的切點
            #timeHeavy_cutoff_start: 新的boundary左邊的候選切點
            #timeHeavy_cutoff_end: 新的boundary右邊的候選切點
            timeHeavy_cutoff = np.array([(timeHeavy[0][i]+timeHeavy[0][i-1])/2 for i in range(1,len(timeHeavy[0]))])
            timeHeavy_cutoff_start = timeHeavy_cutoff[timeHeavy_cutoff<min([min_peak_time_in3,center])][:-3]
            timeHeavy_cutoff_end = timeHeavy_cutoff[timeHeavy_cutoff>max([max_peak_time_in3,center])][3:]

            ###建立boundary左邊、中間、右邊的index
            #within_index: boundary中間的index
            upper_start = [np.array(timeHeavy[i])>start for i in range(len(timeHeavy))]
            lower_end = [np.array(timeHeavy[i])<end for i in range(len(timeHeavy))]
            within_index = [upper_start[i]&lower_end[i] for i in range(len(timeHeavy))]
            del upper_start,lower_end
            #outof_index_pre: boundary左邊的index 
            #outof_index_post: boundary右邊的index 
            outof_index_pre = [np.array(timeHeavy[i])<start for i in range(len(timeHeavy))]
            outof_index_post = [np.array(timeHeavy[i])>end for i in range(len(timeHeavy))]

            #create x-y to plot (中間)
            repeat_within_intHeavy_smooth = [np.array(intHeavy_smooth[i])[list(within_index[i])] for i in range(3)]
            time_diff_within = [np.array(timeHeavy[i])[list(within_index[i])][1]-np.array(timeHeavy[i])[list(within_index[i])][0] for i in range(3)]
            repeat_within_timeHeavy = [list(np.array(timeHeavy[i])[list(within_index[i])]) for i in range(3)]
            
            '''
            #creat x-y to plot (左邊)
            repeat_outof_intHeavy_smooth_pre = [np.array(intHeavy_smooth[i])[list(outof_index_pre[i])] for i in range(3)]
            time_diff_outof_pre = [np.array(timeHeavy[i])[list(outof_index_pre[i])][1]-np.array(timeHeavy[i])[list(outof_index_pre[i])][0] for i in range(3)]
            repeat_outof_timeHeavy_pre = [list(np.array(timeHeavy[i])[list(outof_index_pre[i])]) for i in range(3)]
            #creat x-y to plot (右邊)
            repeat_outof_intHeavy_smooth_post = [np.array(intHeavy_smooth[i])[list(outof_index_post[i])] for i in range(3)]
            time_diff_outof_post = [np.array(timeHeavy[i])[list(outof_index_post[i])][1]-np.array(timeHeavy[i])[list(outof_index_post[i])][0] for i in range(3)]
            repeat_outof_timeHeavy_post = [list(np.array(timeHeavy[i])[list(outof_index_post[i])]) for i in range(3)]
            '''
            
            ############### PLOT-startorigin
            plt.rcParams['figure.figsize'] = (20.0, 15.0)
            plt.rcParams['figure.dpi'] = 150
            colors = ['#0000ff','#a52a2a','#8a2be2']
            
            #建立畫布 3*2
            #Retention_Time-Intensity的圖畫在第3格
            plt.subplot(3,2,3)
            for i,t,c,f in zip(intHeavy_smooth ,timeHeavy,colors,Fragmention):
                plt.plot(t, np.array(i)/100, color=c, linewidth=1, label='%s(heavy)'%f)
            plt.axvline(start, linestyle= '--', linewidth=1,color='black')
            plt.axvline(end, linestyle= '--', linewidth=1,color='black')    
            plt.ylabel('Intensity($10^2$)',fontweight='bold')
            plt.xlabel('Retention Time',fontweight='bold')
            plt.legend()
            plt.title(fn.split('.')[0]+'\n'+ps+'_Heavy',fontweight='bold')
            plt.xticks(fontweight='bold')
            plt.yticks(fontweight='bold')
            #畫框起來的區域之FFT結果，依照Fragmention區分，共三個。
            #畫在畫布的第2,4,6格
            for i in range(len(Fragmention)):
                intensity = list(repeat_within_intHeavy_smooth[i])
                #補零至128
                #intensity = np.array(intensity + [0]*(128-len(intensity)))
                N = len(intensity)
                freq = np.fft.fftfreq(n=N,d=time_diff_within[0])
                intensity_f = np.fft.fft(intensity)
                intensity_a = np.abs(intensity_f/N)
                #訊號後半部是週期性重複，所以之需前一半(1/2)的訊號
                intensity_a_half = intensity_a[:int(len(intensity_a)/2)]
                #peaks: 定義fft後的訊號強度>1的才是有意義訊號，並且忽略掉fft結果頻率為0時的原點訊號。
                peaks = intensity_a_half[intensity_a_half>1][1:] 
                #計算每條peak所佔全部peaks的百分比。
                peaks_pesentage = [j/sum(peaks) for j in peaks]
                sin1_index = list(intensity_a).index(peaks[0])
                sin2_index = list(intensity_a).index(peaks[1])
                
                plt.subplot(3,2,2+i*2)
                plt.stem(freq[:round(len(freq)/2)],intensity_a[:round(len(freq)/2)],use_line_collection=True)
                plt.title('Heavy (%s)'%Fragmention[i], fontsize=15,fontweight='bold')
                plt.grid(True)
                plt.xlabel('Frequency',fontweight='bold')
                plt.ylabel('Amplitude',fontweight='bold')
                plt.xticks(fontweight='bold')
                plt.yticks(fontweight='bold')
                plt.annotate('%.1f%%'%(peaks_pesentage[0]*100), (freq[sin1_index],intensity_a[sin1_index]),color='black')
                plt.annotate('%.1f%%'%(peaks_pesentage[1]*100), (freq[sin2_index],intensity_a[sin2_index]),color='black')

            if '%s_%s_2sin'%(fn,ps) not in os.listdir('./MRM_RT_rearrangement/test'):
                os.mkdir('./MRM_RT_rearrangement/test/%s_%s_2sin'%(fn,ps))
            plt.savefig('./MRM_RT_rearrangement/test/%s_%s_2sin/0.png'%(fn,ps))  
            plt.clf()

            ###########以下開始調整"框"的位置###########
            #判斷好壞基準：只判斷heavy端，"框"所框及的中間區域之fft後前兩個peaks強度之百分比越高越好。
            best_start = start
            best_end = end
            max_2sin = 0
            for i in range(len(Fragmention)):
                intensity = list(repeat_within_intHeavy_smooth[i])
                #補零128
                #intensity = np.array(intensity + [0]*(128-len(intensity)))                
                N = len(intensity)
                intensity_f = np.fft.fft(intensity)
                intensity_a = np.abs(intensity_f/N)
                intensity_a_half = intensity_a[:int(len(intensity_a)/2)]
                peaks = intensity_a_half[intensity_a_half>1][1:] 
                mean = intensity_a_half[intensity_a_half>1][0]
                peaks_pesentage = [j/sum(peaks) for j in peaks]
                #計算前兩個訊號百分比總和
                max_2sin += peaks_pesentage[0] +peaks_pesentage[1] 
            
            PicNumber = 1
            #判斷start還是end先做
            #start: 原本機器框出來的"框"之左邊retention time時間點。
            #end: 原本機器框出來的"框"之右邊retention time時間點。
            #startHight:左端"框"所切到的訊號強度（共3個Fragmention)。
            #endHight:右端"框"所切到的訊號強度（共3個Fragmention)。
            startHight = [(np.array(intHeavy[i])[np.array(timeHeavy[i])<start][-1]+np.array(intHeavy[i])[np.array(timeHeavy[i])>start][0])/2 for i in range(len(timeHeavy))]
            endHight = [(np.array(intHeavy[i])[np.array(timeHeavy[i])<end][-1]+np.array(intHeavy[i])[np.array(timeHeavy[i])>end][0])/2 for i in range(len(timeHeavy))]
            
            #判斷若左端切點訊號強度 高於 右端訊號強度，則start端(左端)先移動切點
            if max(startHight) >= max(endHight): 
                #start: 此時為左端的候補切點
                for start in timeHeavy_cutoff_start:
                    #建立新切點中間的intensity及retention time，分別為repeat_within_intHeavy_smooth,repeat_within_timeHeavy
                    upper_start = [np.array(timeHeavy[i])>start for i in range(len(timeHeavy))]
                    lower_end = [np.array(timeHeavy[i])<end for i in range(len(timeHeavy))]
                    within_index = [upper_start[i]&lower_end[i] for i in range(len(timeHeavy))]
                    del upper_start,lower_end
                    #creat x-y to plot
                    repeat_within_intHeavy_smooth = [np.array(intHeavy_smooth[i])[list(within_index[i])] for i in range(3)]
                    time_diff_within = [np.array(timeHeavy[i])[list(within_index[i])][1]-np.array(timeHeavy[i])[list(within_index[i])][0] for i in range(3)]
                    repeat_within_timeHeavy = [list(np.array(timeHeavy[i])[list(within_index[i])]) for i in range(3)]

                    #FFT
                    sum2sin = 0
                    for i in range(len(Fragmention)):
                        try:
                            intensity = list(repeat_within_intHeavy_smooth[i])
                            #補零128
                            #intensity = np.array(intensity + [0]*(128-len(intensity)))    
                            N = len(intensity)
                            intensity_f = np.fft.fft(intensity)
                            intensity_a = np.abs(intensity_f/N)
                            intensity_a_half = intensity_a[:int(len(intensity_a)/2)]
                            peaks = intensity_a_half[intensity_a_half>1][1:] 
                            mean = intensity_a_half[intensity_a_half>1][0]
                            peaks_pesentage = [j/sum(peaks) for j in peaks]
                            #sum2sin：新切點的前兩個訊號百分比總和
                            sum2sin += peaks_pesentage[0]+peaks_pesentage[1] 
                        except:
                            continue
                    #sum2sin：新切點的前兩個訊號百分比總和
                    #max_2sin:舊切點的前兩個訊號百分比總和
                    #如果新>舊，那就更新切點，並作圖，直到左端候補全部判斷完成
                    if sum2sin > max_2sin:
                        max_2sin = sum2sin
                        best_start = start
                        #print(start)         
                        colors = ['#0000ff','#a52a2a','#8a2be2']
                        plt.subplot(3,2,3)
                        for i,t,c,f in zip(intHeavy_smooth ,timeHeavy,colors,Fragmention):
                            plt.plot(t, np.array(i)/100, color=c, linewidth=1, label='%s(heavy)'%f)
                        plt.axvline(start, linestyle= '--', linewidth=1,color='black')
                        plt.axvline(end, linestyle= '--', linewidth=1,color='black')    
                        plt.ylabel('Intensity($10^2$)',fontweight='bold')
                        plt.xlabel('Retention Time',fontweight='bold')
                        plt.legend()
                        plt.title(fn.split('.')[0]+'\n'+ps+'_Heavy',fontweight='bold')
                        plt.xticks(fontweight='bold')
                        plt.yticks(fontweight='bold')
                        for i in range(len(Fragmention)):
                            intensity = repeat_within_intHeavy_smooth[i]
                            t = repeat_within_timeHeavy[i]
                            N = len(intensity)
                            freq = np.fft.fftfreq(n=N,d=time_diff_within[0])
                            intensity_f = np.fft.fft(intensity)
                            intensity_a = np.abs(intensity_f/N)
                            intensity_a_half = intensity_a[:int(len(intensity_a)/2)]
                            peaks = intensity_a_half[intensity_a_half>1][1:] 
                            peaks_pesentage = [j/sum(peaks) for j in peaks]
                            sin1_index = list(intensity_a).index(peaks[0])
                            sin2_index = list(intensity_a).index(peaks[1])
                            plt.subplot(3,2,2+i*2)
                            plt.stem(freq[:round(len(freq)/2)],intensity_a[:round(len(freq)/2)],use_line_collection=True)
                            plt.title('Heavy (%s)'%Fragmention[i], fontsize=15,fontweight='bold')
                            plt.grid(True)
                            plt.xlabel('Frequency',fontweight='bold')
                            plt.ylabel('Amplitude',fontweight='bold')
                            plt.xticks(fontweight='bold')
                            plt.yticks(fontweight='bold')
                            plt.annotate('%.1f%%'%(peaks_pesentage[0]*100), (freq[sin1_index],intensity_a[sin1_index]),color='black')
                            plt.annotate('%.1f%%'%(peaks_pesentage[1]*100), (freq[sin2_index],intensity_a[sin2_index]),color='black')
                        plt.savefig('./MRM_RT_rearrangement/test/%s_%s_2sin/%d'%(fn,ps,PicNumber))       
                        plt.clf()
                        PicNumber +=1

                #end: 此時end為候補的右端切點
                for end in timeHeavy_cutoff_end[::-1]: #timeHeavy_cutoff_end[::-1]要把end從後面開始往前調
                    #建立新切點中間的intensity及retention time，分別為repeat_within_intHeavy_smooth,repeat_within_timeHeavy
                    upper_start = [np.array(timeHeavy[i])>best_start for i in range(len(timeHeavy))]
                    lower_end = [np.array(timeHeavy[i])<end for i in range(len(timeHeavy))]
                    within_index = [upper_start[i]&lower_end[i] for i in range(len(timeHeavy))]
                    del upper_start,lower_end
                    #creat x-y to plot
                    repeat_within_intHeavy_smooth = [np.array(intHeavy_smooth[i])[list(within_index[i])] for i in range(3)]
                    time_diff_within = [np.array(timeHeavy[i])[list(within_index[i])][1]-np.array(timeHeavy[i])[list(within_index[i])][0] for i in range(3)]
                    repeat_within_timeHeavy = [list(np.array(timeHeavy[i])[list(within_index[i])]) for i in range(3)]

                    #FFT
                    sum2sin = 0
                    for i in range(len(Fragmention)):
                        try:
                            intensity = list(repeat_within_intHeavy_smooth[i])
                            #補零128
                            #intensity = np.array(intensity + [0]*(128-len(intensity)))    
                            N = len(intensity)
                            intensity_f = np.fft.fft(intensity)
                            intensity_a = np.abs(intensity_f/N)
                            intensity_a_half = intensity_a[:int(len(intensity_a)/2)]
                            peaks = intensity_a_half[intensity_a_half>1][1:] 
                            mean = intensity_a_half[intensity_a_half>1][0]
                            peaks_pesentage = [j/sum(peaks) for j in peaks]
                            sum2sin += peaks_pesentage[0] + peaks_pesentage[1] 
                        except:
                            continue

                    if sum2sin > max_2sin:
                        max_2sin = sum2sin
                        best_end = end         
                        colors = ['#0000ff','#a52a2a','#8a2be2']
                        plt.subplot(3,2,3)
                        for i,t,c,f in zip(intHeavy_smooth ,timeHeavy,colors,Fragmention):
                            plt.plot(t, np.array(i)/100, color=c, linewidth=1, label='%s(heavy)'%f)
                        plt.axvline(best_start, linestyle= '--', linewidth=1,color='black')
                        plt.axvline(end, linestyle= '--', linewidth=1,color='black')    
                        plt.ylabel('Intensity($10^2$)',fontweight='bold')
                        plt.xlabel('Retention Time',fontweight='bold')
                        plt.legend()
                        plt.title(fn.split('.')[0]+'\n'+ps+'_Heavy',fontweight='bold')
                        plt.xticks(fontweight='bold')
                        plt.yticks(fontweight='bold')
                        for i in range(len(Fragmention)):
                            intensity = repeat_within_intHeavy_smooth[i]
                            t = repeat_within_timeHeavy[i]
                            N = len(intensity)
                            freq = np.fft.fftfreq(n=N,d=time_diff_within[0])
                            intensity_f = np.fft.fft(intensity)
                            intensity_a = np.abs(intensity_f/N)
                            intensity_a_half = intensity_a[:int(len(intensity_a)/2)]
                            peaks = intensity_a_half[intensity_a_half>1][1:] 
                            peaks_pesentage = [j/sum(peaks) for j in peaks]
                            sin1_index = list(intensity_a).index(peaks[0])
                            sin2_index = list(intensity_a).index(peaks[1])
                            plt.subplot(3,2,2+i*2)
                            plt.stem(freq[:round(len(freq)/2)],intensity_a[:round(len(freq)/2)],use_line_collection=True)
                            plt.title('Heavy (%s)'%Fragmention[i], fontsize=15,fontweight='bold')
                            plt.grid(True)
                            plt.xlabel('Frequency',fontweight='bold')
                            plt.ylabel('Amplitude',fontweight='bold')
                            plt.xticks(fontweight='bold')
                            plt.yticks(fontweight='bold')
                            plt.annotate('%.1f%%'%(peaks_pesentage[0]*100), (freq[sin1_index],intensity_a[sin1_index]),color='black')
                            plt.annotate('%.1f%%'%(peaks_pesentage[1]*100), (freq[sin2_index],intensity_a[sin2_index]),color='black')
                        plt.savefig('./MRM_RT_rearrangement/test/%s_%s_2sin/%d'%(fn,ps,PicNumber))       
                        plt.clf()
                        PicNumber +=1
            
            #若右端先動，則邏輯同上述左端先動之方法。
            else: 
                #end端先
                for end in timeHeavy_cutoff_end[::-1]:
                    upper_start = [np.array(timeHeavy[i])>start for i in range(len(timeHeavy))]
                    lower_end = [np.array(timeHeavy[i])<end for i in range(len(timeHeavy))]
                    within_index = [upper_start[i]&lower_end[i] for i in range(len(timeHeavy))]
                    del upper_start,lower_end
                    #creat x-y to plot
                    repeat_within_intHeavy_smooth = [np.array(intHeavy_smooth[i])[list(within_index[i])] for i in range(3)]
                    time_diff_within = [np.array(timeHeavy[i])[list(within_index[i])][1]-np.array(timeHeavy[i])[list(within_index[i])][0] for i in range(3)]
                    repeat_within_timeHeavy = [list(np.array(timeHeavy[i])[list(within_index[i])]) for i in range(3)]

                    #FFT
                    sum2sin = 0
                    for i in range(len(Fragmention)):
                        try:
                            intensity = list(repeat_within_intHeavy_smooth[i])
                            #補零128
                            #intensity = np.array(intensity + [0]*(128-len(intensity)))  
                            N = len(intensity)
                            intensity_f = np.fft.fft(intensity)
                            intensity_a = np.abs(intensity_f/N)
                            intensity_a_half = intensity_a[:int(len(intensity_a)/2)]
                            peaks = intensity_a_half[intensity_a_half>1][1:] 
                            mean = intensity_a_half[intensity_a_half>1][0]
                            peaks_pesentage = [j/sum(peaks) for j in peaks]
                            sum2sin += peaks_pesentage[0] + peaks_pesentage[1] 
                        except:
                            continue

                    if sum2sin > max_2sin:
                        max_2sin = sum2sin
                        best_end = end         
                        colors = ['#0000ff','#a52a2a','#8a2be2']
                        plt.subplot(3,2,3)
                        for i,t,c,f in zip(intHeavy_smooth ,timeHeavy,colors,Fragmention):
                            plt.plot(t, np.array(i)/100, color=c, linewidth=1, label='%s(heavy)'%f)
                        plt.axvline(start, linestyle= '--', linewidth=1,color='black')
                        plt.axvline(end, linestyle= '--', linewidth=1,color='black')    
                        plt.ylabel('Intensity($10^2$)',fontweight='bold')
                        plt.xlabel('Retention Time',fontweight='bold')
                        plt.legend()
                        plt.title(fn.split('.')[0]+'\n'+ps+'_Heavy'+'%s_%s'%(start,end),fontweight='bold')
                        plt.xticks(fontweight='bold')
                        plt.yticks(fontweight='bold')
                        for i in range(len(Fragmention)):
                            intensity = list(repeat_within_intHeavy_smooth[i])
                            #補零128
                            #intensity = np.array(intensity + [0]*(128-len(intensity)))  
                            N = len(intensity)
                            freq = np.fft.fftfreq(n=N,d=time_diff_within[0])
                            intensity_f = np.fft.fft(intensity)
                            intensity_a = np.abs(intensity_f/N)
                            intensity_a_half = intensity_a[:int(len(intensity_a)/2)]
                            peaks = intensity_a_half[intensity_a_half>1][1:] 
                            peaks_pesentage = [j/sum(peaks) for j in peaks]
                            sin1_index = list(intensity_a).index(peaks[0])
                            sin2_index = list(intensity_a).index(peaks[1])
                            plt.subplot(3,2,2+i*2)
                            plt.stem(freq[:round(len(freq)/2)],intensity_a[:round(len(freq)/2)],use_line_collection=True)
                            plt.title('Heavy (%s)'%Fragmention[i], fontsize=15,fontweight='bold')
                            plt.grid(True)
                            plt.xlabel('Frequency',fontweight='bold')
                            plt.ylabel('Amplitude',fontweight='bold')
                            plt.xticks(fontweight='bold')
                            plt.yticks(fontweight='bold')
                            plt.annotate('%.1f%%'%(peaks_pesentage[0]*100), (freq[sin1_index],intensity_a[sin1_index]),color='black')
                            plt.annotate('%.1f%%'%(peaks_pesentage[1]*100), (freq[sin2_index],intensity_a[sin2_index]),color='black')
                        plt.savefig('./MRM_RT_rearrangement/test/%s_%s_2sin/%d'%(fn,ps,PicNumber))       
                        plt.clf()
                        PicNumber +=1
                        
                #start端後
                for start in timeHeavy_cutoff_start:
                    upper_start = [np.array(timeHeavy[i])>start for i in range(len(timeHeavy))]
                    lower_end = [np.array(timeHeavy[i])<best_end for i in range(len(timeHeavy))]
                    within_index = [upper_start[i]&lower_end[i] for i in range(len(timeHeavy))]
                    del upper_start,lower_end
                    #creat x-y to plot
                    repeat_within_intHeavy_smooth = [np.array(intHeavy_smooth[i])[list(within_index[i])] for i in range(3)]
                    time_diff_within = [np.array(timeHeavy[i])[list(within_index[i])][1]-np.array(timeHeavy[i])[list(within_index[i])][0] for i in range(3)]
                    repeat_within_timeHeavy = [list(np.array(timeHeavy[i])[list(within_index[i])]) for i in range(3)]

                    #FFT
                    sum2sin = 0
                    for i in range(len(Fragmention)):
                        try:
                            intensity = list(repeat_within_intHeavy_smooth[i])
                            #補零128
                            #intensity = np.array(intensity + [0]*(128-len(intensity)))  
                            t = repeat_within_timeHeavy[i]
                            N = len(intensity)
                            intensity_f = np.fft.fft(intensity)
                            intensity_a = np.abs(intensity_f/N)
                            intensity_a_half = intensity_a[:int(len(intensity_a)/2)]
                            peaks = intensity_a_half[intensity_a_half>1][1:] 
                            mean = intensity_a_half[intensity_a_half>1][0]
                            peaks_pesentage = [j/sum(peaks) for j in peaks]
                            sum2sin += peaks_pesentage[0]+peaks_pesentage[1] 
                        except:
                            continue

                    if sum2sin > max_2sin:
                        max_2sin = sum2sin
                        best_start = start
                        #print(start)         
                        colors = ['#0000ff','#a52a2a','#8a2be2']
                        plt.subplot(3,2,3)
                        for i,t,c,f in zip(intHeavy_smooth ,timeHeavy,colors,Fragmention):
                            plt.plot(t, np.array(i)/100, color=c, linewidth=1, label='%s(heavy)'%f)
                        plt.axvline(start, linestyle= '--', linewidth=1,color='black')
                        plt.axvline(best_end, linestyle= '--', linewidth=1,color='black')    
                        plt.ylabel('Intensity($10^2$)',fontweight='bold')
                        plt.xlabel('Retention Time',fontweight='bold')
                        plt.legend()
                        plt.title(fn.split('.')[0]+'\n'+ps+'_Heavy'+'%s_%s'%(start,end),fontweight='bold')
                        plt.xticks(fontweight='bold')
                        plt.yticks(fontweight='bold')
                        for i in range(len(Fragmention)):
                            intensity = repeat_within_intHeavy_smooth[i]
                            t = repeat_within_timeHeavy[i]
                            N = len(intensity)
                            freq = np.fft.fftfreq(n=N,d=time_diff_within[0])
                            intensity_f = np.fft.fft(intensity)
                            intensity_a = np.abs(intensity_f/N)
                            intensity_a_half = intensity_a[:int(len(intensity_a)/2)]
                            peaks = intensity_a_half[intensity_a_half>1][1:] 
                            peaks_pesentage = [j/sum(peaks) for j in peaks]
                            sin1_index = list(intensity_a).index(peaks[0])
                            sin2_index = list(intensity_a).index(peaks[1])
                            plt.subplot(3,2,2+i*2)
                            plt.stem(freq[:round(len(freq)/2)],intensity_a[:round(len(freq)/2)],use_line_collection=True)
                            plt.title('Heavy (%s)'%Fragmention[i], fontsize=15,fontweight='bold')
                            plt.grid(True)
                            plt.xlabel('Frequency',fontweight='bold')
                            plt.ylabel('Amplitude',fontweight='bold')
                            plt.xticks(fontweight='bold')
                            plt.yticks(fontweight='bold')
                            plt.annotate('%.1f%%'%(peaks_pesentage[0]*100), (freq[sin1_index],intensity_a[sin1_index]),color='black')
                            plt.annotate('%.1f%%'%(peaks_pesentage[1]*100), (freq[sin2_index],intensity_a[sin2_index]),color='black')                            
                        plt.savefig('./MRM_RT_rearrangement/test/%s_%s_2sin/%d'%(fn,ps,PicNumber))       
                        plt.clf()
                        PicNumber +=1
        except:
            print(fn,ps) #找不到的資料列印出來
